## Importação dos pacotes

In [23]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

## Declaração da função de Rastrigin

In [22]:
def rastrigin(x):
    return 10*len(x) + sum([(xi**2 - 10 * np.cos(2 * np.pi * xi)) for xi in x])

![rastr.png](./assets/rastr.png)

![rastr2.png](./assets/rastr2.png)

A função Rastrigin possui vários mínimos locais. É altamente multimodal, mas as localizações dos mínimos são distribuídas de forma regular. Ela é mostrada no gráfico acima em sua forma bidimensional.

Disponível na íntegra em: https://www.sfu.ca/~ssurjano/rastr.html

- Multimodalidade: Possui múltiplos mínimos locais, desafiando algoritmos a encontrar o mínimo global.

- Complexidade do Espaço de Busca: Tem uma superfície ondulada e complexa, simulando problemas de otimização reais.

- Avaliação da Robustez dos Algoritmos: Testa a capacidade dos algoritmos de evitar mínimos locais e explorar o espaço de busca.

- Relevância para Problemas Reais: Representa melhor a complexidade dos problemas de otimização do mundo real.

- Testes de Otimização Global: Adequada para avaliar algoritmos de otimização global, como PSO, que precisam lidar com múltiplos mínimos locais.


## Declaração da classe das Partículas

In [24]:
class Particle:
    def __init__(self, bounds):
        self.position = np.random.uniform(bounds[:, 0], bounds[:, 1])
        self.velocity = np.random.uniform(-1, 1, size=(bounds.shape[0],))
        self.best_position = np.copy(self.position)
        self.best_score = rastrigin(self.position)

## Declaração da classe do PSO em si

In [26]:
class PSO:
    def __init__(self, n_particles, bounds, iterations, c1=2.05, c2=2.05):  # soma de c1 e c2 até 4.1
        self.n_particles = n_particles  # número de partículas na população.
        self.bounds = bounds  # limites do espaço de busca
        self.iterations = iterations  # número de iterações
        self.c1 = c1
        self.c2 = c2  # coeficientes de aceleração que controlam o comportamento cognitivo e social das partículas.
        self.particles = [Particle(bounds) for _ in range(n_particles)]  # lista das partículas
        self.gbest_score = np.inf  # melhor valor da função objetivo encontrado globalmente.
        self.gbest_position = np.zeros(bounds.shape[0])  # melhor posição encontrada globalmente.
        self.w = np.linspace(0.9, 0.1, self.iterations)  # fator de inércia que diminui linearmente ao longo das iterações.
        self.best_scores = []  # Lista que armazena o melhor score de cada iteração
        self.positions_over_time = []  # Lista que armazena as posições das partículas ao longo do tempo para animação

    def optimize(self):
        for i in range(self.iterations):
            positions = []
            for particle in self.particles:
                fitness = rastrigin(particle.position)  # calcula o valor da função Rastrigin na posição atual da partícula

                if fitness < particle.best_score:  # atualiza a melhor posição da partícula se o fitness atual for melhor
                    particle.best_score = fitness
                    particle.best_position = np.copy(particle.position)

                if fitness < self.gbest_score:  # atualiza a melhor posição global se o fitness atual for o melhor encontrado até agora
                    self.gbest_score = fitness
                    self.gbest_position = np.copy(particle.position)

                r1 = np.random.random()  # números aleatórios para a parte estocástica das componentes cognitiva e social
                r2 = np.random.random() 

                cognitive = self.c1 * r1 * (particle.best_position - particle.position)  # componente cognitiva que atrai a partícula para sua melhor posição pessoal
                social = self.c2 * r2 * (self.gbest_position - particle.position)  # componente social que atrai a partícula para a melhor posição global

                particle.velocity = self.w[i] * particle.velocity + cognitive + social  # atualização da velocidade da partícula com base no cognitivo e social
                particle.position += particle.velocity

                particle.position = np.clip(particle.position, self.bounds[:, 0], self.bounds[:, 1])  # garante que a nova posição esteja dentro dos limites permitidos
                positions.append(particle.position)

            self.positions_over_time.append(positions)
            self.best_scores.append(self.gbest_score)

        return self.gbest_position, self.gbest_score

    def animate(self):
        fig, ax = plt.subplots()

        # escolhendo ateatoriamente 2 dimensões para plotar
        dim1, dim2 = np.random.choice(range(self.bounds.shape[0]), size=2, replace=False)

        def update(num):
            ax.clear()
            ax.set_title(f'Iteração {num}')
            ax.set_xlim([self.bounds[dim1, 0], self.bounds[dim1, 1]])
            ax.set_ylim([self.bounds[dim2, 0], self.bounds[dim2, 1]])
            positions = self.positions_over_time[num]
            for position in positions:
                ax.scatter(position[dim1], position[dim2])

        ani = animation.FuncAnimation(fig, update, frames=self.iterations, repeat=False)
        writervideo = animation.FFMpegWriter(fps=2)
        ani.save('pso.mp4', writer=writervideo)
        plt.show()

def main():
    bounds = np.array([[-5.12, 5.12], [-5.12, 5.12], [-5.12, 5.12]])
    pso = PSO(n_particles=30, bounds=bounds, iterations=100)
    best_position, best_score = pso.optimize()
    print(f'Melhor posição: {best_position}, Melhor score: {best_score}')
    # pso.animate()  # descomentar aqui caso rodar no Colab

main()

Melhor posição: [ 1.56193602e-08 -9.94958622e-01 -1.44159289e-08], Melhor score: 0.9949590570934319


## Feito pelo Chat-GPT
"Faça um algoritmo de pso em python"

In [3]:
import numpy as np

# Função objetivo (esférica)
def objective_function(x):
    return np.sum(x ** 2)

# Algoritmo PSO
def pso(num_particles, num_dimensions, num_iterations, bounds, c1=2.0, c2=2.0, w=0.5):
    # Inicialização
    positions = np.random.uniform(bounds[0], bounds[1], (num_particles, num_dimensions))
    velocities = np.random.uniform(-1, 1, (num_particles, num_dimensions))
    pbest_positions = np.copy(positions)
    pbest_scores = np.apply_along_axis(objective_function, 1, pbest_positions)
    gbest_position = pbest_positions[np.argmin(pbest_scores)]
    gbest_score = np.min(pbest_scores)

    # Iterações
    for iteration in range(num_iterations):
        for i in range(num_particles):
            # Atualizar velocidade
            r1, r2 = np.random.rand(2)
            velocities[i] = (w * velocities[i] +
                             c1 * r1 * (pbest_positions[i] - positions[i]) +
                             c2 * r2 * (gbest_position - positions[i]))

            # Atualizar posição
            positions[i] = positions[i] + velocities[i]

            # Confinar posições dentro dos limites
            positions[i] = np.clip(positions[i], bounds[0], bounds[1])

            # Avaliar nova posição
            score = objective_function(positions[i])

            # Atualizar pbest
            if score < pbest_scores[i]:
                pbest_positions[i] = positions[i]
                pbest_scores[i] = score

            # Atualizar gbest
            if score < gbest_score:
                gbest_position = positions[i]
                gbest_score = score

        print(f"Iteração {iteration + 1}/{num_iterations} - Melhor Score: {gbest_score}")

    return gbest_position, gbest_score

# Parâmetros do PSO
num_particles = 30
num_dimensions = 2
num_iterations = 100
bounds = (-10, 10)

# Execução do PSO
best_position, best_score = pso(num_particles, num_dimensions, num_iterations, bounds)

print(f"Melhor posição: {best_position}")
print(f"Melhor score: {best_score}")


Iteração 1/100 - Melhor Score: 0.18002502978213286
Iteração 2/100 - Melhor Score: 0.12743803819547414
Iteração 3/100 - Melhor Score: 0.12743803819547414
Iteração 4/100 - Melhor Score: 0.04278453852730482
Iteração 5/100 - Melhor Score: 0.006854813444553724
Iteração 6/100 - Melhor Score: 0.006854813444553724
Iteração 7/100 - Melhor Score: 0.006269109898233389
Iteração 8/100 - Melhor Score: 0.006269109898233389
Iteração 9/100 - Melhor Score: 0.006269109898233389
Iteração 10/100 - Melhor Score: 0.006269109898233389
Iteração 11/100 - Melhor Score: 0.006001532980520965
Iteração 12/100 - Melhor Score: 0.0005496629994841132
Iteração 13/100 - Melhor Score: 8.309819158696208e-05
Iteração 14/100 - Melhor Score: 8.309819158696208e-05
Iteração 15/100 - Melhor Score: 8.309819158696208e-05
Iteração 16/100 - Melhor Score: 8.309819158696208e-05
Iteração 17/100 - Melhor Score: 8.309819158696208e-05
Iteração 18/100 - Melhor Score: 8.309819158696208e-05
Iteração 19/100 - Melhor Score: 8.309819158696208e-0